In [1]:
import pandas as pd

In [4]:
df = pd.read_csv('MIEM.csv')
df.head()

,number,title,type
0,1.0,Изучение транспорта заряженных коллоидных част...,НИР
1,2.0,Изучение конвекции Марангони и транспорта колл...,НИР
2,3.0,Разработка программного комплекса решения эндш...,НИР
3,4.0,Программа объединения точек платежей для проти...,Прогр.
4,5.0,Антифрод (antifraud),Прогр.


In [7]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/anastasiaibaeva/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
from pymystem3 import Mystem
mystem = Mystem() 

In [18]:
import string

punctuation = string.punctuation

In [19]:
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text

In [20]:
from tqdm import tqdm

tqdm.pandas()
df['processed'] = df['title'].progress_apply(preprocess_text)

100%|██████████| 253/253 [00:00<00:00, 1294.79it/s]


In [21]:
df.head()

,number,title,type,processed
0,1.0,Изучение транспорта заряженных коллоидных част...,НИР,изучение транспорт заряжать коллоидный частица...
1,2.0,Изучение конвекции Марангони и транспорта колл...,НИР,изучение конвекция марангонь транспорт коллоид...
2,3.0,Разработка программного комплекса решения эндш...,НИР,разработка программный комплекс решение эндшпи...
3,4.0,Программа объединения точек платежей для проти...,Прогр.,программа объединение точка платеж противодейс...
4,5.0,Антифрод (antifraud),Прогр.,антифрод antifraud


In [22]:
df['concat'] = df['processed'] + ' ' + df['type']
df.head()

,number,title,type,processed,concat
0,1.0,Изучение транспорта заряженных коллоидных част...,НИР,изучение транспорт заряжать коллоидный частица...,изучение транспорт заряжать коллоидный частица...
1,2.0,Изучение конвекции Марангони и транспорта колл...,НИР,изучение конвекция марангонь транспорт коллоид...,изучение конвекция марангонь транспорт коллоид...
2,3.0,Разработка программного комплекса решения эндш...,НИР,разработка программный комплекс решение эндшпи...,разработка программный комплекс решение эндшпи...
3,4.0,Программа объединения точек платежей для проти...,Прогр.,программа объединение точка платеж противодейс...,программа объединение точка платеж противодейс...
4,5.0,Антифрод (antifraud),Прогр.,антифрод antifraud,антифрод antifraud Прогр.


In [24]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

card_docs = [TaggedDocument(doc.split(' '), [i]) for i, doc in enumerate(df.concat)]

In [25]:
model = Doc2Vec(vector_size=64, min_count=1, epochs=20)
model.build_vocab(card_docs)

In [26]:
model.train(card_docs, total_examples=model.corpus_count, epochs=model.epochs)

In [27]:
card2vec = [model.infer_vector((df['concat'][i].split(' '))) for i in range(0,len(df['concat']))]

In [28]:
import numpy as np

dtv = np.array(card2vec).tolist()
df['vec'] = dtv
df.head()

,number,title,type,processed,concat,vec
0,1.0,Изучение транспорта заряженных коллоидных част...,НИР,изучение транспорт заряжать коллоидный частица...,изучение транспорт заряжать коллоидный частица...,"[-0.0070869713090360165, -0.013778778724372387..."
1,2.0,Изучение конвекции Марангони и транспорта колл...,НИР,изучение конвекция марангонь транспорт коллоид...,изучение конвекция марангонь транспорт коллоид...,"[-0.010782599449157715, -0.022596212103962898,..."
2,3.0,Разработка программного комплекса решения эндш...,НИР,разработка программный комплекс решение эндшпи...,разработка программный комплекс решение эндшпи...,"[-0.005868301726877689, -0.01714327372610569, ..."
3,4.0,Программа объединения точек платежей для проти...,Прогр.,программа объединение точка платеж противодейс...,программа объединение точка платеж противодейс...,"[-0.00847938284277916, -0.006686461623758078, ..."
4,5.0,Антифрод (antifraud),Прогр.,антифрод antifraud,антифрод antifraud Прогр.,"[-0.008986253291368484, 0.002366742817685008, ..."


In [29]:
df.to_csv('miem_vectors.csv', index=False) 